<a href="https://colab.research.google.com/github/JahedNaghipoor/Machine-Learning/blob/master/Advanced_Scikit_Learn_Tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load Python Package
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
# Load data (loading Titanic dataset)
data  = pd.read_csv('https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv')

In [ ]:
# Set X and y
X = data.drop('Survived',axis=1)
y = data[['Survived']]

In [ ]:
# Make Transformer using clumn_selector
preprocessing = make_column_transformer(
    (OneHotEncoder(), make_column_selector(dtype_include='object')),
    (SimpleImputer(), make_column_selector(dtype_include='int')),
    remainder='drop'
)

In [ ]:
# Fit-Transform data with transformer
preprocessing.fit_transform(data)

<887x893 sparse matrix of type '<class 'numpy.float64'>'
	with 3499 stored elements in Compressed Sparse Row format>

# Make pipeline


In [ ]:
# Split Train, Val and Test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

In [ ]:
# Print dataFrames size
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(531, 7)
(178, 7)
(178, 7)


In [ ]:
# Set variables
ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
imputer = SimpleImputer(add_indicator=True, verbose=1)
scaler = StandardScaler()
clf = LogisticRegression()

In [ ]:
# Make Transformer
preprocessing = make_column_transformer(
(make_pipeline(imputer,scaler),['Age','Siblings/Spouses Aboard','Parents/Children Aboard','Fare'])
,(ohe, ['Pclass','Sex','Name'])
,remainder='passthrough')

In [ ]:
# Make pipeline
pipe = make_pipeline(preprocessing, clf)

In [ ]:
# Fit model
pipe.fit(X_train, y_train.values.ravel())
print("Best score : %f" % pipe.score(X_test, y_test.values.ravel()))

Best score : 0.786517


In [ ]:
# Cross-validation
print("Cross-validation score : %f" % cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean())

Cross-validation score : 0.793690


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

#optimal tuning parameters using GridSearch

In [ ]:
# Set params for Grid Search
params = {}
params['logisticregression__C'] = [0.001,0.1,0.2,0.3,0.4]
params['logisticregression__max_iter'] = [100,200,500]

In [ ]:
# Run grid search
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X,y.values.ravel())
print("Best score : %f" % grid.best_score_)

Best score : 0.794795


In [ ]:
print(grid.best_params_)

{'logisticregression__C': 0.1, 'logisticregression__max_iter': 100}
